In [95]:

import os, re, sys
import numpy as np

filename = "data.txt"
with open(filename, 'r') as f:
    s = f.read()
    initial, compuatations = s.split("\n\n")


calculated = {x.split(": ")[0]: bool(int(x.split(": ")[1])) for x in initial.split("\n")}
compuatations = compuatations.split("\n")



outcomes = []
dependencies = []
operations = []

for compuatation in compuatations:
    args, outcome = compuatation.split(" -> ")
    args = args.split(" ")

    dependencies.append((args[0], args[2]))
    operations.append(args[1])
    outcomes.append(outcome)



#print(outcomes)
for i in range(0, 100):
    if 'z' + ["0", ''][i >= 10] + str(i) not in outcomes:
        max_z = i
        break

for i in range(0, 100):
    if 'x' + ["0", ''][i >= 10] + str(i) not in initial:
        max_x = i
        break

for i in range(0, 100):
    if 'y' + ["0", ''][i >= 10] + str(i) not in initial:
        max_y = i
        break

zs = ['z' + ["0", ''][i >= 10] + str(i) for i in range(max_z)]
ys = ['y' + ["0", ''][i >= 10] + str(i) for i in range(max_y)]
xs = ['x' + ["0", ''][i >= 10] + str(i) for i in range(max_x)]

print(zs)
print(ys)
print(xs)

['z00', 'z01', 'z02', 'z03', 'z04', 'z05', 'z06', 'z07', 'z08', 'z09', 'z10', 'z11', 'z12', 'z13', 'z14', 'z15', 'z16', 'z17', 'z18', 'z19', 'z20', 'z21', 'z22', 'z23', 'z24', 'z25', 'z26', 'z27', 'z28', 'z29', 'z30', 'z31', 'z32', 'z33', 'z34', 'z35', 'z36', 'z37', 'z38', 'z39', 'z40', 'z41', 'z42', 'z43', 'z44', 'z45']
['y00', 'y01', 'y02', 'y03', 'y04', 'y05', 'y06', 'y07', 'y08', 'y09', 'y10', 'y11', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y18', 'y19', 'y20', 'y21', 'y22', 'y23', 'y24', 'y25', 'y26', 'y27', 'y28', 'y29', 'y30', 'y31', 'y32', 'y33', 'y34', 'y35', 'y36', 'y37', 'y38', 'y39', 'y40', 'y41', 'y42', 'y43', 'y44']
['x00', 'x01', 'x02', 'x03', 'x04', 'x05', 'x06', 'x07', 'x08', 'x09', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44']


In [96]:

def get_dependencies(name):
    for i in range(len(outcomes)):
        if outcomes[i] == name:
            a, b = dependencies[i] 
            deps = set()
            if a[0] in ['x', 'y']:
                deps.add(a)
            else:
                deps |= get_dependencies(a)


            if b[0] in ['x', 'y']:
                deps.add(b)
            else:
                deps |= get_dependencies(b)

            return deps



print(get_dependencies("z05"))


{'x02', 'y05', 'x05', 'x03', 'y00', 'x00', 'x01', 'y01', 'y02', 'y04', 'x04', 'y03'}


In [97]:


def get_dependencies(name):
    deps = set()
    for i in range(len(outcomes)):
        if outcomes[i] == name:
            a, b = dependencies[i] 
            

            #print(name + " is  the result of " + a + " "+ operations[i] + " " + b)

            if a[0] in ['x', 'y']:
                deps.add(a)
            else:
                deps |= get_dependencies(a)


            if b[0] in ['x', 'y']:
                deps.add(b)
            else:
                deps |= get_dependencies(b)

            return deps
    return deps


def get_influences_below_i(name, max_i): # depends on get_dependencies and dependencies and outcomes
    infs = set()
    for i in range(len(dependencies)):
        a, b = dependencies[i]
        c = outcomes[i]
        

        if name == a:
            deps = get_dependencies(b) | {b}

            for dep in deps:
                if dep[0] in ['x', 'y'] and int(dep[1:]) > max_i:
                    break # b somehow depends on an x and y bigger than max_i -> dont use it
            else:
                infs = infs | {c} | get_influences_below_i(c, max_i)

        if name == b:
            deps = get_dependencies(a) | {a}

            for dep in deps:
                if dep[0] in ['x', 'y'] and int(dep[1:]) > max_i:
                    break # b somehow depends on an x and y bigger than max_i -> dont use it
            else:
                infs = infs | {c} | get_influences_below_i(c, max_i)
    
    return infs
#print(get_dependencies('x01'))

print(get_influences_below_i('x01', 1))
print(get_influences_below_i('y01', 1))



{'pgc', 'kjn', 'z01', 'hjc', 'hth'}
{'pgc', 'kjn', 'z01', 'hjc', 'hth'}


In [98]:

affects = {}
for i in range(len(dependencies)):
    a, b = dependencies[i]
    op = operations[i]
    c = outcomes[i]
    if affects.get(a):
        affects[a].append(c)
    else:
        affects[a] = [c] 

    if affects.get(b):
        affects[b].append(c)
    else:
        affects[b] = [c] 


def comp(a, b, op):
    if op == 'XOR':
        return a ^ b
    elif op == 'OR':
        return a | b
    elif op == 'AND':
        return a & b


def compute(local_results, pos): # uses dependencies
    prev_l = -1
    #print("trying to compute, ", pos, " with ", local_results)
    while True:
        l = len(local_results)
        if not l > prev_l:
            print(local_results)
            print("wtf")
            return 5
            #sys.exit()

        prev_l = l
        if pos in local_results:
            return local_results[pos]

        for i in range(len(dependencies)):
            a, b = dependencies[i]
            if a in local_results and b in local_results:
                a, b = local_results[a], local_results[b]
                local_results[outcomes[i]] = comp(a, b, operations[i])

        
def find_carry_and_z(x, y, c, possible_cz): # uses compute
    poss_start_values = [ \
        {x: 0, y: 0, c:0}, 
        {x: 0, y: 0, c:1},
        {x: 0, y: 1, c:0},
        {x: 0, y: 1, c:1},
        {x: 1, y: 0, c:0},
        {x: 1, y: 0, c:1},
        {x: 1, y: 1, c:0},
        {x: 1, y: 1, c:1},
    ]
    corr_results_c = [
        0, 0, 0, 1, 0, 1, 1, 1
    ]
    corr_results_z = [
        0, 1, 1, 0, 1, 0, 0, 1
    ]

    poss_carry_bits = []
    poss_outc_bits = []
    for cz in possible_cz:
        ress = []
        for i in range(8):
            start_values = poss_start_values[i].copy()
            ress.append(compute(start_values, cz))
            
        if ress == corr_results_c:
            poss_carry_bits.append(cz)
        elif ress == corr_results_z:
            poss_outc_bits.append(cz)

    return poss_carry_bits, poss_outc_bits



In [99]:

def swap(x, y):

    if y in affects.get(x, []) or x in affects.get(y, []):
        return # it is impossible to swap these

    # we also need to modify affectss 
    print("swapping ", x, " and ", y)
    x_i = None
    y_i = None
    for i in range(len(dependencies)):
        a, b = dependencies[i]
        c = outcomes[i]

        

        if x == c:
            x_i = i
            print("case 1", a, b, c)
            print("af a", affects[a])
            print("af b", affects[b])
            affects[a].remove(x)
            affects[b].remove(x)
            affects[a].append(y)
            affects[b].append(y)

        if y == c:
            y_i = i
            print("case 2", a, b, c)
            print("af a", affects[a])
            print("af b", affects[b])
            affects[a].remove(y)
            affects[b].remove(y)
            affects[a].append(x)
            affects[b].append(x)
    
    outcomes[x_i] = y
    outcomes[y_i] = x





seen_elems = set()


outputs = ["z00"]
carries = ["skt"]
prev_carry = "skt" # the first carry bit
all_swaps = []
for i in range(1, 45):
    print('')
    x, y, z = xs[i], ys[i], zs[i]
    
    inf_x = get_influences_below_i(x, i)
    inf_y = get_influences_below_i(y, i)

    poss_cz = (inf_x | inf_y) - seen_elems # all the elements that depend solely on x00, x01, y00 and y01
    seen_elems = seen_elems | inf_x | inf_y
    
    c_poss, z_poss = find_carry_and_z(x, y, carries[-1], poss_cz)
    print(c_poss, z_poss)


    if z in z_poss and len(c_poss) >= 1:
        carries.append(c_poss[-1])
        outputs.append(z)
    else:
        print("\n" * 5 + "oh no")
        solved = False
        poss_cz = list(poss_cz)

        for i in range(len(poss_cz)):
            cz1 = poss_cz[i]
            for j in range(i+1, len(poss_cz)):
                cz2 = poss_cz[j]

                print("we try this swap; ", cz1, cz2)

                swap(cz1, cz2)
                c_poss, z_poss = find_carry_and_z(x, y, carries[-1], poss_cz)

                if z in z_poss and len(c_poss) >= 1:
                    carries.append(c_poss[-1])
                    outputs.append(z)
                    all_swaps.append(cz1)
                    all_swaps.append(cz2)
                    solved = True
                
                swap(cz1, cz2) # swap them back
                
                

            if solved:
                break


print(carries, outputs)

    

    # carry bit c = ?



['pgc'] ['z01']

['wdm'] ['z02']

['rkm'] ['z03']

['rdj'] ['z04']

['wvr'] ['z05']

['z06'] ['fkp']





oh no
we try this swap;  jgw ghf
we try this swap;  jgw fkp
we try this swap;  jgw z06
swapping  jgw  and  z06
case 2 bpp ghf z06
af a ['z06']
af b ['z06']
case 1 x06 y06 jgw
af a ['bpp', 'jgw']
af b ['bpp', 'jgw']
{'x06': 0, 'y06': 0, 'wvr': 0, 'bpp': 0, 'z06': 0}
wtf
{'x06': 0, 'y06': 0, 'wvr': 1, 'bpp': 0, 'z06': 0}
wtf
{'x06': 0, 'y06': 1, 'wvr': 0, 'bpp': 0, 'z06': 1}
wtf
{'x06': 0, 'y06': 1, 'wvr': 1, 'bpp': 0, 'z06': 1}
wtf
{'x06': 1, 'y06': 0, 'wvr': 0, 'bpp': 0, 'z06': 1}
wtf
{'x06': 1, 'y06': 0, 'wvr': 1, 'bpp': 0, 'z06': 1}
wtf
{'x06': 1, 'y06': 1, 'wvr': 0, 'bpp': 1, 'z06': 0}
wtf
{'x06': 1, 'y06': 1, 'wvr': 1, 'bpp': 1, 'z06': 0}
wtf
{'x06': 0, 'y06': 0, 'wvr': 0, 'bpp': 0, 'z06': 0}
wtf
{'x06': 0, 'y06': 0, 'wvr': 1, 'bpp': 0, 'z06': 0}
wtf
{'x06': 0, 'y06': 1, 'wvr': 0, 'bpp': 0, 'z06': 1}
wtf
{'x06': 0, 'y06': 1, 'wvr': 1, 'bpp': 0, 'z06': 1}
wtf
{'x06': 1, 'y06': 

In [103]:
print(all_swaps)

print(",".join(sorted(all_swaps)))

['fkp', 'z06', 'z11', 'ngr', 'z31', 'mfm', 'krj', 'bpt']
bpt,fkp,krj,mfm,ngr,z06,z11,z31
